In [1]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\Working_attributes")
from Imports import * 

In [2]:
today = '2024-10-31'

print_to_excel = False
excel_tofolder_on_Z = False

In [25]:
base = 'RISKACCESS'
base = 'xxwh'
export_data = export_from_WHWEEK(f'''select * 
from {base}."unifiedOcp"
where "sourceTable" = 'sapBwInvestmentReport'
and "reportDate" = TO_DATE('{today}', 'YYYY-MM-DD')
--fetch first 10 rows only''')
export_data.shape

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:176: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


(37032, 40)

In [28]:
data_work = export_data.copy(deep=True) # copy export data
data_work[['Завод','Вид_проекта','Версия_035','Версия_045','Версия_075','Версия_085']] = np.nan #add empty cols
# work with cols
data_work['Проект'] = data_work['recordId'].str.split('-').str[0]
data_work.maturityDateTo = data_work.maturityDateTo.astype(str).apply(lambda x: '-'.join([x[:-3],'01']))
data_work.maturityDateTo = pd.to_datetime(data_work.maturityDateTo)
data_work['Календарный_год'] = data_work['maturityDateTo'].dt.year
data_work['Календарный_месяц'] = data_work['maturityDateTo'].dt.month
data_work = data_work[['holding','ocpSegment','salesUnitsId','Завод','recordId','counterpartyCode','counterpartySegment','Вид_проекта','accountSubtype','Проект','accountCode','maturityDateTo','Календарный_год','Календарный_месяц','effectiveCurrency','Версия_035','Версия_045','Версия_075','Версия_085','volumeNotionalCurrency','usdRate','volumeUsd']]
data_work = data_work.rename(columns={'recordId':'СПП-элемент','accountSubtype':'Профиль_проекта','accountCode':'Вид_затрат'})

In [5]:
# Аня, в файлик по инвестициям надо добавить еще одну страницу, куда положить агрегированный набор данных по сокращенному списку полей. 
# Поля следующие. По расчётным полям amount поставить фильтр <>0
pivot_data = data_work[data_work.volumeNotionalCurrency!=0]
pivot_data = pivot_data.groupby(as_index=False, by=['holding','salesUnitsId','ocpSegment','effectiveCurrency','Календарный_год','Календарный_месяц','counterpartySegment','maturityDateTo'])[['volumeNotionalCurrency','volumeUsd']].agg(sum)

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_4080\3280017617.py:4: FutureWarning: The provided callable <built-in function sum> is currently using DataFrameGroupBy.sum. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "sum" instead.
  pivot_data = pivot_data.groupby(as_index=False, by=['holding','salesUnitsId','ocpSegment','effectiveCurrency','Календарный_год','Календарный_месяц','counterpartySegment','maturityDateTo'])[['volumeNotionalCurrency','volumeUsd']].agg(sum)


In [6]:
### to excel
for holding in ['EUROCHEM', 'SUEK']:
    data = 0
    if print_to_excel == True:
        Output_file = f'{str(date.today())}_Invest_nv_{holding}.xlsx'
        if excel_tofolder_on_Z == True:
            Output_path = 'z:\\Anna_Klimova\\OCP\\Archive\\'
            Output_file = Output_path + Output_file
        data = data_work[data_work.holding==holding]
        data.to_excel(Output_file, sheet_name='Invest', index=False)
        new_list(pivot_data[pivot_data.holding==holding], Output_file, sheet_name='pivot')

Проверка counterpartySegment

In [ ]:
# introgroupCounterpartyCode
# если upper("mdgBuNameIdMapping"."buName") = upper("unifiedOcp"."counterpartyName"), то "mdgBuNameIdMapping"."salesUnitId"

# counterpartySegment
# если "unifiedOcp"."introgroupCounterpartyCode" is null , тогда 'External'
# иначе xxmdg."salesUnits"."OCPsegment"  по связи  "unifiedOcp"."introgroupCounterpartyCode" = xxmdg."salesUnits"."id"

In [14]:
cty_data = export_data[['counterpartyName','introgroupCounterpartyCode','counterpartySegment']].drop_duplicates(subset='counterpartyName').reset_index(drop=True)
cty_data['cty_code_Anna'] = merge_Mapping(cty_data, 'counterpartyName')
cty_data['cty_segment_Anna'] = merge_SalesUnits(cty_data, 'cty_code_Anna', merge_col='ocpSegment')
cty_data

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['SUEK RU' 'SUEK RU' 'SUEK RU' 'SUEK RU' 'SUEK RU' 'SUEK RU' 'SUEK RU'
 'SUEK RU' 'SGC' 'SGC' 'RUFERT' 'SUEK RU' 'SUEK RU' 'SUEK RU' 'NTC'
 'SUEK RU' 'SGC' 'SGC' 'NTC' 'SGC' 'SGC' 'RUFERT' 'NTC' 'RUFERT' 'RUFERT'
 'RUFERT' 'NTC' 'RUFERT' 'RUFERT' 'RUFERT' 'RUFERT' 'RUFERT' 'TRADING'
 'SGC' 'RUFERT' 'RUFERT' 'SGC' 'SGC' 'SGC' 'SGC' 'SGC' 'SGC']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data.loc[merge_

,counterpartyName,introgroupCounterpartyCode,counterpartySegment,cty_code_Anna,cty_segment_Anna
0,"ООО ""Офис-Проджект""",1100,SUEK RU,External,External
1,"ООО ""ЦентрЭксим""",1100,SUEK RU,External,External
2,"ООО ""Пожтехпроект""",1100,SUEK RU,External,External
3,"ООО ""Номари Сиайэс""",1100,SUEK RU,External,External
4,"ООО ""НеваСитиСервис""",3200,SUEK RU,External,External
...,...,...,...,...,...
4407,"ООО Фирма ""ПАЛИТРА",RUEMO,RUFERT,External,External
4408,"ТОО ""Expo",KZKRT,RUFERT,External,External
4409,"ТОО ""DigiTech Group""",KZKRT,RUFERT,External,External
4410,"АО ""Кселл""",None,External,External,External


In [35]:
data_work_new = export_data.copy().reset_index(drop=True)
print(data_work_new.shape[0])
data_work_new['cty_code_Anna'] = merge_Mapping(data_work_new, 'counterpartyName')
data_work_new['cty_segment_Anna'] = merge_SalesUnits(data_work_new, 'cty_code_Anna', merge_col='ocpSegment')
data_work_new[['counterpartyName','cty_code_Anna','cty_segment_Anna']]
data_work_new['cty_codes_eq'] = data_work_new.cty_code_Anna	== data_work_new.introgroupCounterpartyCode
data_work_new = data_work_new.sort_values('counterpartyName')

37032


C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:92: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\Working_attributes\Defs.py:33: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['SUEK RU' 'SUEK RU' 'SUEK RU' ... 'SUEK RU' 'SUEK RU' 'SUEK RU']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', 'last_merge'] = merge_data.loc[merge_data[f'{id_colmn}_merge'] != 'External', f'{id_colmn}_merge']


In [37]:
data_work_new.to_excel('cty_check.xlsx')